In [13]:
from turtle import distance
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import open3d as o3d
from scipy import stats
from scipy.spatial.distance import directed_hausdorff
%matplotlib qt

In [30]:
map_reference = cv.imread("../data/map/carla_map_reference_scaled.png")
rows, cols,_ = map_reference.shape
window_name = 'image'

plt.imshow(map_reference)
#map_reference [0:20, 0:20] = (0,0,255)
#cv.imshow(window_name, map_reference)
#cv.waitKey(0) 
#cv.destroyAllWindows() 

In [31]:
map_reference.shape

(169, 300, 3)

In [32]:
world_points = np.array([np.array([-106.686, -4.847]),np.array([20.452, 13.196]),np.array([-64.644, 24.471]),np.array([-27.022, 69.714])])
image_points = np.full_like(world_points)
# in bgr notation
first = (0,0,255)
second = (0,255,0)
third = (255,0,0)
fourth = (0,255,255)

for y in range(rows): 
    for x in range(cols): 
        if (map_reference[y,x] == first): 
            image_points[0] = np.array([x,y])


255 0 0
255 0 0
255 0 0
255 0 0
